In [ ]:
import numpy as np
import random
import pandas as pd


In [ ]:
istanbul_metro = {
    "M2": [
        "YENIKAPI", "VEZNECILER", "HALIC", "SISHANE", "TAKSIM",
        "OSMANBEY", "SISLI", "GAYRETTEPE", "LEVENT", "4 LEVENT",
        "SANAYI MAH.", "ITU", "ATATURK OTO SANAYI", "DARUSSAFAKA",
        "HACIOSMAN"
    ],
    "M3": [
        "BAKIRKOY IDO", "OZGURLUK MEYDANI", "INCIRLI", "HAZNEDAR", "ILKYUVA",
        "MOLLA GURANI", "KIRAZLI", "YENI MAHALLE", "MAHMUTBEY", "ISTOC",
        "IKITELLI SANAYI", "TURGUT OZAL", "SITELER", "BASAK KONUTLARI", "METROKENT",
        "ONURKENT", "SEHIR HASTANESI", "TOPLU KONUTLAR", "KAYASEHIR MERKEZ"
],
    "M4": [
        "KADIKOY", "AYRILIKCESME", "ACIBADEM", "UNALAN", "GOZTEPE",
        "YENISAHRA", "KOZYATAGI", "BOSTANCI", "KUCUKYALI", "MALTEPE",
        "HUZUREVI", "GULSUYU", "ESENKENT CEVIZLI", "HASTANE", "SOGANLIK",
        "KARTAL", "YAKACIK", "PENDIK", "TAVSANTEPE", "FEVZI CAKMAK",
        "YAYALAR", "KURTKOY", "SABIHA GOKCEN HAVALIMANI",
    ],
    # "M5": [
    #     "USKUDAR", "FISTIKAGACI", "BAGLARBASI", "ALTUNIZADE", "KISIKLI",
    #     "BULGURLU", "UMRANIYE", "CARSI", "YAMANEVLER", "CAKMAK",
    #     "IHLAMURKUYU", "ALTINSEHIR", "IMAM HATIP LISESI", "DUDULLU", "NECIP FAZIL",
    #     "CEKMEKOY", "MECLIS", "SARIGAZI", "SANCAKTEPE", "SAMANDIRA MERKEZ"
    # ],
    "M6": [
        "LEVENT  KONKORS", "NISPETIYE", "ETILER", "BOGAZICI"
    ],
    "M7": [
        "YILDIZ", "FULYA", "MECIDIYEKOY", "CAGLAYAN", "KAGITHANE",
        "NURTEPE", "ALIBEYKOY", "CIRCIR", "VEYSELKARANI", "YESILPINAR",
        "KAZIMKARABEKIR", "YENIMAHALLE", "KARADENIZ MAH.", "TEKSTILKENT", "ORUCREIS",
        "GOZTEPE", "MAHMUTBEY"
    ],
    # "M8": [
    #     "BOSTANCI", "EMIN ALI PASA", "AYSE KADIN", "ICERENKOY", "KUCUKBAKKALKOY", "KAYISDAGI",
    #     "MEVLANA", "IMES", "MODOKO KEYAP", "DUDULLU", "HUZUR", "PARSELLER"
    # ],
    "M9": [
        "ATAKOY", "YENIBOSNA", "COBANCESME", "29 EKIM CUMHURIYETIHLAS YUVA", "SANAYI", "MIMAR SINAN",
        "15 TEMMUZ", "HALKALI", "ATATURK MAHALLESI", "BAHARIYE", "MASKO", 
        "IKITELLI SANAYI", "ZIYA GOKALP MAH", "OLIMPIYAT"
    ],
    "M11": [
        "GAYRETTEPE", "KAGITHANE", "HASDAL", "KEMERBURGAZ", "GOKTURK", "IHSANIYE",
        "ISTANBUL HAVALIMANI", "KARGO TERMINALI", "TASOLUK", "ARNAVUTKOY HASTANE"
    ],
    "M11": [
        "GAYRETTEPE", "KAGITHANE", "HASDAL", "KEMERBURGAZ", "GOKTURK", "IHSANIYE",
        "ISTANBUL HAVALIMANI", "KARGO TERMINALI", "TASOLUK", "ARNAVUTKOY HASTANE"
    ],

    "T5": [
        "EMINONU", "KUCUKPAZAR", "CIBALI", "FENER", "BALAT",
        "AYVANSARAY", "FESHANE", "EYUP TELEFERIK", "EYUP DEVLET HASTANESI", "SILAHTARAGA",
        "UNIVERSITE", "ALIBEYKOY", "ALIBEYKOY METRO", "CEP OTOGAR"],

    "T4": [
        "TOPKAPI", "FETIHKAPI", "VATAN", "EDIRNEKAPI", "SEHITLIK",
        "DEMIRKAPI", "TOPCULAR", "RAMI", "BEREC", "SAGMALCILAR",
        "CUKURCESME", "ALI FUAT BASGIL", "TASKOPRU", "KARADENIZ MAHALLESI", "KIPTAS VENEZIA",
        "CUMHURIYET", "BASTABYA", "HACI SUKRU", "YENIMAHALLE", "SULTANCIFTLIGI",
        "CEBECI", "MESCIDI SELAM"],

    "T1": [ 
        "KABATAS", "FINDIKLI", "TOPHANE", "KARAKOY", "EMINONU",
        "SIRKECI", "GULHANE", "SULTANAHMET", "CEMBERLITAS", "BEYAZIT",
        "LALELI", "AKSARAY", "YUSUFPASA", "HASEKI", "FINDIKZADE",
        "CAPA", "PAZARTEKKE", "TOPKAPI", "ATATURK OGRENCI YURDU", "MERKEZEFENDI",
        "AKSEMSETTIN", "MITHATPASA", "ZEYTINBURNU", "MEHMETAKIF", "KERESTECILER",
        "GUNGOREN", "AKINCILAR", "SOGANLI", "CAMI", "GUNESTEPE",
        "BAGCILAR"]
}


In [ ]:
data = pd.read_csv('data/hourly_transportation_202409.csv')
data = data[data["line_name"].isin(["M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9", "T1", "T4", "T5"])]
data = data[["transition_date", "transition_hour", "line_name", "number_of_passage", "station_poi_desc_cd"]]
data = data.rename(columns={"transition_date": "date", "transition_hour": "hour", "line_name": "line", "number_of_passage": "passenger_count", "station_poi_desc_cd": "station"})
data = data[data["hour"] >= 6]

data['station'] = data['station'].str.replace(r'\(.*?\)', '', regex=True)
# Remove unwanted words (KUZEY, GUNEY, DOGU, BATI) and numbers from the 'station' column
data['station'] = data['station'].str.replace(r'\b(HOL|KUZEY|GUNEY|DOGU|BATI)\b', '', regex=True)
data['station'] = data['station'].str.replace(r'\b(T4|M3|M7|M9)\b', '', regex=True)
#remove all digits and numbers at the end of the string
data['station'] = data['station'].str.replace(r'\s*\d+\s*$', '', regex=True)
data['station'] = data['station'].str.replace(r'[\-/]', '', regex=True)  # Remove symbols: (), -, /
data['station'] = data['station'].str.strip()
data["station"] = data["station"].astype(str)

data = data[~data["station"].str.contains("SEYRANTEPE", na=False)]

data = data.groupby(['date', 'hour', 'line', 'station'], as_index=False)['passenger_count'].sum()
data = data.groupby(['hour', 'line', 'station'], as_index=False).agg({'passenger_count': 'mean'})

data


In [ ]:
from collections import OrderedDict

def sort_passenger_data_by_station(data, station_order):
    sorted_data = {}

    for line, hours in data.items():
        if line not in station_order:
            print(f"[UYARI] '{line}' hattı için durak sırası tanımlı değil.")
            continue

        line_order = station_order[line]
        all_stations_in_data = set()
        for hour_data in hours.values():
            all_stations_in_data.update(hour_data.keys())

        # 1. station_order'da olup dict'te olmayanlar
        for station in line_order:
            if station not in all_stations_in_data:
                print(f"[Eksik Veri] '{line}' hattında sıralamada olan '{station}' istasyonu veride yok.")

        # 2. dict'te olup station_order'da olmayanlar
        for station in all_stations_in_data:
            if station not in line_order:
                print(f"[Sıralama Bilgisi Yok] '{line}' hattında veride olan '{station}' istasyonu sıralama listesinde yok.")

        # Asıl sıralama işlemi
        sorted_data[line] = {}
        for hour, stations in hours.items():
            sorted_stations = OrderedDict()
            for station in line_order:
                if station in stations:
                    sorted_stations[station] = stations[station]
            sorted_data[line][hour] = sorted_stations

    return sorted_data


In [ ]:
dict = {}
for i in range(len(data)):
    if data["line"][i] not in dict:
        dict[data["line"][i]] = {}
    if data["hour"][i] not in dict[data["line"][i]]:
        dict[data["line"][i]][data["hour"][i]] = {}
    if data["station"][i] not in dict[data["line"][i]][data["hour"][i]]:
        dict[data["line"][i]][data["hour"][i]][data["station"][i]] = {}
    dict[data["line"][i]][data["hour"][i]][data["station"][i]] = data["passenger_count"][i]
dict

In [ ]:
sorted_data = sort_passenger_data_by_station(dict, istanbul_metro)

In [ ]:
df_rows = []
for line, hours in sorted_data.items():
    for hour, stations in hours.items():
        for station, count in stations.items():
            df_rows.append({
                "line": line, "hour": hour, "station": station, "passenger_count": count
            })

data_sorted = pd.DataFrame(df_rows)
data_sorted

In [ ]:
#cumulative sum of passenger_count shifted by 1 station
data_sorted['cum_passenger_count'] = data_sorted.groupby(['hour', 'line'])['passenger_count'].cumsum()
data_sorted['cum_passenger_count'] = data_sorted.groupby(['hour', 'line'])['cum_passenger_count'].shift(fill_value=0)
data_sorted

In [ ]:
def calculate_passenger_count(data, hour, line, station):
    data = data[(data["hour"] == hour) & (data["line"] == line)]
    station_data = data[data["station"] == station]
    passenger_count = station_data["passenger_count"].iloc[0]
    total_passenger_count = sum(data["passenger_count"])
    cumsum_passenger_count = station_data["cum_passenger_count"].iloc[0]
    
    direction_2 = passenger_count * cumsum_passenger_count / (total_passenger_count-passenger_count)
    direction_1 = passenger_count-direction_2
    return direction_1, direction_2

In [ ]:
data_sorted['direction_1'] = None
data_sorted['direction_2'] = None


for index, row in data_sorted.iterrows():
    direction_1, direction_2 = calculate_passenger_count(data_sorted, row['hour'], row['line'], row['station'])
    data_sorted.at[index, 'direction_1'] = direction_1
    data_sorted.at[index, 'direction_2'] = direction_2
data_sorted['direction_1'] = pd.to_numeric(data_sorted['direction_1'], errors='coerce')
data_sorted['direction_2'] = pd.to_numeric(data_sorted['direction_2'], errors='coerce')
data_sorted

In [ ]:
import pandas as pd

df_orig = data_sorted.copy()  # orijinal DataFrame

new_rows = []

for line in df_orig['line'].unique():
    df_line = df_orig[df_orig['line'] == line].copy()

    for hour, group in df_line.groupby('hour'):
        # Sıralı hale getir
        group_sorted = group.sort_values(by='station', key=lambda s: s.factorize()[0])

        # Orijinal yön (direction_1)
        df_orig_dir = group_sorted.copy()
        df_orig_dir['in_passenger'] = df_orig_dir['direction_1']
        df_orig_dir = df_orig_dir.drop(columns=['direction_2'])

        # Ters yön (direction_2)
        df_rev_dir = group_sorted.iloc[::-1].copy()
        df_rev_dir['line'] = f"{line}-R"
        df_rev_dir['in_passenger'] = df_rev_dir['direction_2']
        df_rev_dir = df_rev_dir.drop(columns=['direction_1'])


        new_rows.extend([df_orig_dir, df_rev_dir])
         


df_split = pd.concat(new_rows, ignore_index=True)
df_split = df_split.drop(columns=["direction_1", "direction_2", "cum_passenger_count", "passenger_count"])
df_split

In [ ]:
#cumulative sum of passenger_count shifted by 1 station
df_split['cum_passenger_count'] = df_split.groupby(['hour', 'line'])['in_passenger'].cumsum()
df_split['cum_passenger_count'] = df_split.groupby(['hour', 'line'])['cum_passenger_count'].shift(fill_value=0)
df_split

In [ ]:
def calculate_out_passenger_count(data, hour, line, station):
    data = data[(data["hour"] == hour) & (data["line"] == line)]
    station_data = data[data["station"] == station]
    passenger_count = station_data["in_passenger"].iloc[0]
    total_passenger_count = sum(data["in_passenger"])
    cumsum_passenger_count = station_data["cum_passenger_count"].iloc[0]
    out = passenger_count * cumsum_passenger_count / (total_passenger_count-passenger_count)
    return out


In [ ]:
df_split['out_passenger'] = None


for index, row in df_split.iterrows():
    out = calculate_out_passenger_count(df_split, row['hour'], row['line'], row['station'])
    df_split.at[index, 'out_passenger'] = out

df_split['out_passenger'] = pd.to_numeric(df_split['out_passenger'], errors='coerce')
df_split["current_passenger"] = df_split["cum_passenger_count"] - df_split["out_passenger"] + df_split["in_passenger"]
df_split

In [ ]:
data = df_split.copy()
dict = {}
for i in range(len(data)):
    if data["line"][i] not in dict:
        dict[data["line"][i]] = {}
    if data["hour"][i] not in dict[data["line"][i]]:
        dict[data["line"][i]][data["hour"][i]] = {}
    if data["station"][i] not in dict[data["line"][i]][data["hour"][i]]:
        dict[data["line"][i]][data["hour"][i]][data["station"][i]] = {}
    dict[data["line"][i]][data["hour"][i]][data["station"][i]]["in_passenger"] = float(data["in_passenger"][i])
    dict[data["line"][i]][data["hour"][i]][data["station"][i]]["cum_passenger_count"] = float(data["cum_passenger_count"][i])
    dict[data["line"][i]][data["hour"][i]][data["station"][i]]["out_passenger"] = float(data["out_passenger"][i])
    dict[data["line"][i]][data["hour"][i]][data["station"][i]]["current_passenger"] = float(data["current_passenger"][i])

dict

In [ ]:
length_df = pd.read_excel(r"data\rayl-sistemlere-ait-hat-uzunluk-bilgileri.xlsx")
length_sum = length_df.groupby(length_df.columns[0])[length_df.columns[-1]].sum().reset_index()
length_sum[length_sum.columns[-1]] = length_sum[length_sum.columns[-1]].astype(float) / 1000
length_sum = length_sum.rename(columns={length_sum.columns[-1]: "length(km)"})
lines = istanbul_metro.keys()
#drop the lines that are not in the istanbul_metro
length_sum = length_sum[length_sum[length_sum.columns[0]].isin(lines)]
length_sum

In [14]:
df_split['out_passenger'] = None

for index, row in df_split.iterrows():
    out = calculate_out_passenger_count(df_split, row['hour'], row['line'], row['station'])
    df_split.at[index, 'out_passenger'] = out

df_split['out_passenger'] = pd.to_numeric(df_split['out_passenger'], errors='coerce')
df_split["current_passenger"] = df_split["cum_passenger_count"] - df_split["out_passenger"] + df_split["in_passenger"]
df_split

,line,hour,station,in_passenger,cum_passenger_count,out_passenger,current_passenger
0,M2,6,YENIKAPI,1198.250000,0.000000,0.000000,1198.250000
1,M2,6,VEZNECILER,76.931960,1198.250000,33.553563,1241.628397
2,M2,6,HALIC,73.505744,1275.181960,34.075056,1314.612648
3,M2,6,SISHANE,153.935685,1348.687704,77.746613,1424.876776
4,M2,6,TAKSIM,238.817291,1502.623389,138.795570,1602.645110
...,...,...,...,...,...,...,...
5719,T5-R,23,BALAT,24.085976,331.864853,18.727333,337.223495
5720,T5-R,23,FENER,25.284440,355.950828,21.145353,360.089915
5721,T5-R,23,CIBALI,30.405205,381.235268,27.565720,384.074753
5722,T5-R,23,KUCUKPAZAR,39.270216,411.640473,39.270216,411.640473


In [15]:
length_df = pd.read_excel(r"data\rayl-sistemlere-ait-hat-uzunluk-bilgileri.xlsx")
length_sum = length_df.groupby(length_df.columns[0])[length_df.columns[-1]].sum().reset_index()
length_sum[length_sum.columns[-1]] = length_sum[length_sum.columns[-1]].astype(float) / 1000
length_sum = length_sum.rename(columns={length_sum.columns[-1]: "length(km)"})
lines = istanbul_metro.keys()
#drop the lines that are not in the istanbul_metro
length_sum = length_sum[length_sum[length_sum.columns[0]].isin(lines)]
length_sum

FileNotFoundError: [Errno 2] No such file or directory: 'data\\rayl-sistemlere-ait-hat-uzunluk-bilgileri.xlsx'

In [ ]:
TL = 4.47884
POPULATION_SIZE = 50
OPERATIONAL_COST = 1000
COT = 115
GENERATIONS = 100
FREQ_BOUNDS = (2, 10) 
STOP_TIME_BOUNDS = (20, 90)  # Seconds

density_cost_w, standing_cost_w, time_w  = 3.0, 1.5, 115/60

In [ ]:
energy_df = pd.read_excel(r"data\metro-hatlar-enerji-tuketimi.xlsx")
energy_df.drop(energy_df.columns[-1], axis=1, inplace=True)
energy_df.columns = energy_df.iloc[0]
energy_df = energy_df[1:].reset_index(drop=True)
energy_df = energy_df[energy_df["Hat"].isin(istanbul_metro.keys())]
energy_df

In [ ]:
capacity_df = pd.read_csv(r"data\hat_ortalamalari.csv")
capacity_df = capacity_df[capacity_df["Hat"].isin(istanbul_metro.keys())]
capacity_df = capacity_df.rename(columns={"Hat": "line"})
capacity_df["total_capacity"] = capacity_df["koltuk"] + capacity_df["ayakta"]
capacity_df

In [ ]:
#create a dictionary for energy consumption and length
line_info = {}
for index, row in energy_df.iterrows():
    line = row["Hat"]
    line_info[line] = {
        "energy": float(row["100 KM'de Enerji Tüketimi (kWh)"]),
        "length": length_sum[length_sum[length_sum.columns[0]] == line][length_sum.columns[-1]].values[0]
    }
#calculate the energy consumption for each line
for line in line_info.keys():
    line_info[line]["energy_cost"] = line_info[line]["energy"] * line_info[line]["length"] / 100 *TL

# add koltuk ayakta and total capacity to the line_info dictionary
for index, row in capacity_df.iterrows():
    line = row["line"]
    line_info[line]["koltuk"] = float(row["koltuk"])
    line_info[line]["ayakta"] = float(row["ayakta"])
    line_info[line]["total_capacity"] = line_info[line]["koltuk"] + line_info[line]["ayakta"]




line_info

In [ ]:
def cost_function(frequency, stop_time, line_data,line_info,line_name, time_w, density_cost_w, standing_cost_w):
    # print(type(line_data), type(line_info), type(line_name), type(time_w), type(density_cost_w), type(standing_cost_w))
    # print(line_data)
    op_cost = 0
    wait_cost = 0
    unserved_cost = 0
    density_cost = 0
    standing_cost = 0
    i = 0
    unserved_cost_w = time_w * 3

    for _, hour in line_data.items():
        # print("hour", type(hour))
        # print(hour)
        op_cost += frequency * line_info[line_name]["energy_cost"]
        for _, station in hour.items():
            # print("station", type(station))
            # print(station)
            #print(type(station["in_passenger"]), type(stop_time), type(frequency), type(station["cum_passenger_count"]))
            wait_cost += (station["in_passenger"] * (60 / frequency)/2) * time_w +  ((station["cum_passenger_count"] + station["in_passenger"]) * stop_time / 60) * time_w
            unserved_cost += max(0,(station["in_passenger"] - max(0,(line_info[line_name]["total_capacity"] - station["cum_passenger_count"])))) * (60 / frequency) * unserved_cost_w 
            density_cost += station["cum_passenger_count"] / line_info[line_name]["total_capacity"] * density_cost_w 
            standing_cost += max(0,(station["cum_passenger_count"] - line_info[line_name]["koltuk"])) * standing_cost_w
        i += 1

    return (op_cost + wait_cost + unserved_cost + density_cost + standing_cost)/i , op_cost/i, wait_cost/i, unserved_cost/i, density_cost/i, standing_cost/i


In [ ]:
def initialize_population(pop_size, freq_bounds, stop_time_bounds):
    population = []
    for _ in range(int(pop_size)):
        freq = random.uniform(*freq_bounds)
        stop_time = random.uniform(*stop_time_bounds)
        population.append([freq, stop_time])
    return population


In [ ]:
def evaluate_population(population, line_data, line_info, line_name, w2, w3, w4):
    fitness_values = []
    op = []
    wait = []
    unserved = []
    density = []
    standing = []

    for individual in population:
        freq, stop_time = individual
        cost, op_cost, wait_cost, unserved_cost, density_cost, standing_cost = cost_function(freq, stop_time, line_data, line_info, line_name, w2, w3, w4)
        fitness_values.append(cost)
        op.append(op_cost)
        wait.append(wait_cost)
        unserved.append(unserved_cost)
        density.append(density_cost)
        standing.append(standing_cost)
    return fitness_values, op, wait, unserved, density, standing


In [ ]:
def tournament_selection(population, fitness_values, tournament_size=3):
    selected = []
    pop_size = len(population)
    for _ in range(pop_size):
        contenders_idx = random.sample(range(pop_size), tournament_size)
        contenders_fitness = [fitness_values[i] for i in contenders_idx]
        winner_idx = contenders_idx[np.argmin(contenders_fitness)]
        selected.append(population[winner_idx])
    return selected


In [ ]:
def crossover(parent1, parent2):
    # Uniform crossover
    child1, child2 = [], []
    for gene1, gene2 in zip(parent1, parent2):
        if random.random() < 0.5:
            child1.append(gene1)
            child2.append(gene2)
        else:
            child1.append(gene2)
            child2.append(gene1)
    return child1, child2


In [ ]:
def mutate(individual, freq_bounds, stop_time_bounds, mutation_rate=0.1, mutation_scale=0.1):
    # Mutate frequency
    if random.random() < mutation_rate:
        mutation = random.uniform(-mutation_scale, mutation_scale) * (freq_bounds[1] - freq_bounds[0])
        individual[0] = np.clip(individual[0] + mutation, *freq_bounds)
    # Mutate stop_time
    if random.random() < mutation_rate:
        mutation = random.uniform(-mutation_scale, mutation_scale) * (stop_time_bounds[1] - stop_time_bounds[0])
        individual[1] = np.clip(individual[1] + mutation, *stop_time_bounds)
    return individual


In [ ]:
def genetic_algorithm(line_data, line_info, line_name, 
                      pop_size=POPULATION_SIZE, generations=GENERATIONS, 
                      freq_bounds=FREQ_BOUNDS, stop_time_bounds=STOP_TIME_BOUNDS,
                      w2=density_cost_w, w3=standing_cost_w, w4=time_w):

    population = initialize_population(pop_size, freq_bounds, stop_time_bounds)
    
    best_solution = None
    best_fitness = float('inf')
    
    for gen in range(generations):
        fitness_values, op, wait, unserved, density, standing = evaluate_population(population, line_data, line_info, line_name, w2, w3, w4)

        # Track best solution
        min_idx = np.argmin(fitness_values)
        if fitness_values[min_idx] < best_fitness:
            best_fitness = fitness_values[min_idx]
            best_solution = population[min_idx]
            best_op = op[min_idx]
            best_wait = wait[min_idx]
            best_unserved = unserved[min_idx]
            best_density = density[min_idx]
            best_standing = standing[min_idx]
            
        
        print(f"Generation {gen+1}, Best Cost: {best_fitness:.4f}, Best Operational Cost: {best_op:.4f}, "
              f"Best Wait Cost: {best_wait:.4f}, Best Unserved Cost: {best_unserved:.4f}, Best Density Cost: {best_density:.4f}, "
              f"Best Standing Cost: {best_standing:.4f}, Frequency: {best_solution[0]:.2f}, Stop Time: {best_solution[1]:.2f}")

        # Selection
        selected = tournament_selection(population, fitness_values)

        # Create next generation
        next_population = []
        for i in range(0, pop_size, 2):
            parent1 = selected[i]
            parent2 = selected[(i+1) % pop_size]
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, freq_bounds, stop_time_bounds)
            child2 = mutate(child2, freq_bounds, stop_time_bounds)
            next_population.extend([child1, child2])
        
        population = next_population[:pop_size]  # Ensure population size

    return best_solution, best_fitness


In [ ]:
# Example usage (for line "M2", replace with your actual line data):
best_params, best_cost = genetic_algorithm(
    dict["M2"], line_info, "M2", w2=density_cost_w, w3=standing_cost_w, w4=time_w
)
print(f"Best frequency: {best_params[0]}, Best stop time: {best_params[1]}, Cost: {best_cost}")
